In [2]:
import numpy as np 
import astropy.constants as c
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import math
#list all the constants used
G=c.G.value
M_sun=c.M_sun.value
M_earth=c.M_earth.value
R_sun=c.R_sun.value
R_earth=c.R_earth.value
hour=3600 #s
day=3600*24
e=0.1
t_LC=1765.5 #s

def delta(Rp, Rs):
    delta = (Rp*R_earth / (Rs*R_sun))**2*1e6
    return delta

def SNR(delta,n,sigma_s):
    SNR=delta*n**0.5/sigma_s
    return SNR

def sigma_s(t, sigma_LC, cdppindex): #t/hour
    t_LC=1765.5/hour 

    return sigma_LC*(t/t_LC)**cdppindex

def t_dur(P,Rs,e,a): #hour
    t_dur = P*day*Rs*R_sun*(1-e**2)/(np.pi*a)/hour
    return t_dur

def a(Ms,P):
    a=((G*Ms*M_sun*(P*day)**2)/(4*np.pi**2))**(1/3)
    return a 

def n(t_obs,P): # the unite of t_obs and P is day
    n=t_obs/P
    return n

def f_n(t_obs,P):
    if t_obs <=2*P:
        return 0
    if t_obs> 2*P and t_obs< 3*P:
        return (t_obs/P-2)
    if t_obs>=3*P:
        return 1
    
def f_eff(SNR):
    if SNR <=6:
        return 0
    if SNR >6 and SNR<=12:
        return (SNR-6)/6
    if SNR>12:
        return 1
    
def f_geo(Rp,Rs,a,e):
    f_geo = (Rp*R_earth+Rs*R_sun)/(a*(1-e**2))
    return f_geo

In [2]:
kepler_q17=pd.read_csv('kepler_steller_17.csv',sep='|')

In [3]:
kepler_q17.columns

Index(['kepid', 'tm_designation', 'teff', 'teff_err1', 'teff_err2', 'logg',
       'logg_err1', 'logg_err2', 'feh', 'feh_err1', 'feh_err2', 'mass',
       'mass_err1', 'mass_err2', 'st_radius', 'radius_err1', 'radius_err2',
       'dens', 'dens_err1', 'dens_err2', 'prov_sec', 'kepmag', 'dist',
       'dist_err1', 'dist_err2', 'nconfp', 'nkoi', 'ntce', 'datalink_dvr',
       'st_delivname', 'st_vet_date_str', 'degree_ra', 'degree_dec',
       'st_quarters', 'teff_prov', 'logg_prov', 'feh_prov', 'jmag', 'jmag_err',
       'hmag', 'hmag_err', 'kmag', 'kmag_err', 'dutycycle', 'dataspan',
       'mesthres01p5', 'mesthres02p0', 'mesthres02p5', 'mesthres03p0',
       'mesthres03p5', 'mesthres04p5', 'mesthres05p0', 'mesthres06p0',
       'mesthres07p5', 'mesthres09p0', 'mesthres10p5', 'mesthres12p0',
       'mesthres12p5', 'mesthres15p0', 'rrmscdpp01p5', 'rrmscdpp02p0',
       'rrmscdpp02p5', 'rrmscdpp03p0', 'rrmscdpp03p5', 'rrmscdpp04p5',
       'rrmscdpp05p0', 'rrmscdpp06p0', 'rrmscdpp07p5',

In [4]:
kepler_q17['st_quarters']

0         11111111111111111
1         11111111111111111
2         11111111111111111
3         11111111111111111
4         11111111111111111
5         11111111111111111
6         11111111111111111
7         11100000000000000
8         11111111111111111
9         11111111111111111
10        11111111111111111
11        11111111111111111
12        11111111111111111
13        11111111111111111
14        11111111111111111
15        11111111111111111
16           11111111111111
17        11111111111111111
18        11100100010000000
19           11111111111111
20        11111111111111111
21           11111111111111
22        11111111111111111
23        11111111111111111
24         1111111111111111
25           11111111111111
26        11111111111111111
27           11111111111111
28        11111111111111111
29        11111111111111111
                ...        
200008    11111111111111111
200009    11111111111111111
200010    11111111111111111
200011    11111011101110111
200012    1111101110

In [18]:
len(str(kepler_q17['st_quarters'][1]))

#get the observing in every quarter of kepler survey
tq1=54997.48122-54964.01099
tq2=55090.96492-55002.01748
tq3=55181.99660-55092.72221
tq4=55274.70384-55184.87774
tq5=55370.66003-55275.99115
tq6=55461.79386-55371.94733
tq7=55552.04909-55462.67251
tq8=55634.84602-55567.86468
tq9=55738.42395-55641.01696
tq10=55832.76587-55739.34343
tq11=55930.82669-55833.70579
tq12=56014.52273-55931.90966
tq13=56105.55441-56015.23787
tq14=56203.81957-56106.63736
tq15=56303.63768-56205.98550
tq16=56390.46005-56304.59804
tq17=56423.50115-56391.72690

tq_list=[]
tq_list.append(tq1)
tq_list.append(tq2)
tq_list.append(tq3)
tq_list.append(tq4)
tq_list.append(tq5)
tq_list.append(tq6)
tq_list.append(tq7)
tq_list.append(tq8)
tq_list.append(tq9)
tq_list.append(tq10)
tq_list.append(tq11)
tq_list.append(tq12)
tq_list.append(tq13)
tq_list.append(tq14)
tq_list.append(tq15)
tq_list.append(tq16)
tq_list.append(tq17)

t_obs=[]
for i in range(0,len(kepler_q17)):
    temp=str(kepler_q17['st_quarters'][i])
    time=0
    if len(temp)==17:
        for j in range(0,len(temp)):
            if temp[j]=='1':
                time+=tq_list[j]
    else:
        for j in range(0,len(temp)):
            delta=int(17-len(temp))
            if temp[j]=='1':
                time+=tq_list[j+delta]
                
    t_obs.append(time)
        

In [22]:
t_obs.index(0)

52

In [23]:
kepler_q17['st_quarters'][52]

0

In [26]:
with open('data/kepler_stellar_q17_2020.csv','w') as f:
    s=['kepid',
       'ra',
       'dec',
       'cdpp3',
       'cdpp6',
       'cdpp12',
       'st_quarters',
       't_obs',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(kepler_q17)):
        #select the data with cdpp
        if kepler_q17['rrmscdpp03p0'][i]==kepler_q17['rrmscdpp03p0'][i] and kepler_q17['rrmscdpp06p0'][i]==kepler_q17['rrmscdpp06p0'][i] and kepler_q17['rrmscdpp12p0'][i]==kepler_q17['rrmscdpp12p0'][i]:
            writer.writerow({'kepid':kepler_q17['kepid'][i],
                            'ra':kepler_q17['degree_ra'][i],
                            'dec':kepler_q17['degree_dec'][i],
                            'cdpp3':kepler_q17['rrmscdpp03p0'][i],
                            'cdpp6':kepler_q17['rrmscdpp06p0'][i],
                            'cdpp12':kepler_q17['rrmscdpp12p0'][i],
                            'st_quarters':kepler_q17['st_quarters'][i],
                            't_obs':t_obs[i]})

In [27]:
stellar=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_2020.csv')
print(stellar)

        angDist     kepid         ra        dec    cdpp3    cdpp6   cdpp12  \
0           0.0   6362386  291.26416  41.748840  132.305  127.382  124.423   
1           0.0   8555437  290.11371  44.607410  192.360  149.855  123.781   
2           0.0  12102994  286.54398  50.602310  885.947  670.090  510.271   
3           0.0  10136766  290.15582  47.192989  205.128  151.758  111.203   
4           0.0   2441862  291.10583  37.761162  155.849  128.596  112.388   
5           0.0   6362395  291.26816  41.719559  223.063  187.579  177.446   
6           0.0   8555473  290.12607  44.659081  131.376  101.292   82.939   
7           0.0  12103027  286.57404  50.620010  373.880  287.891  224.798   
8           0.0  10136837  290.18085  47.133839  167.130  123.841   92.214   
9           0.0   6362396  291.26859  41.780010  124.296  130.847  145.577   
10          0.0   8555546  290.15100  44.614601  142.850  105.586   79.934   
11          0.0  12103067  286.60526  50.600929  245.386  200.37

In [29]:
count=0
for i in range(0,len(stellar)):
    if stellar['kepid'][i]==stellar['kepid.1'][i]:
        count+=1

print(count)
        

113942


In [30]:
data=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_2020.csv')
with open('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        'st_quarters',
        't_obs',
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['kepid'][i]==data['kepid.1'][i]:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            'st_quarters':data['st_quarters'][i],
                            't_obs':data['t_obs'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            })
            
    


In [44]:
data=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_2020.csv')
print(len(data))

113942


In [45]:
with open('data/kepler_stellar_q17_add_dispersions_reduction_reduction_hot_2020.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        'st_quarters',
        't_obs',
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['sigma_vra'][i]>=2 or data['sigma_vdec'][i]>=2:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            'st_quarters':data['st_quarters'][i],
                            't_obs':data['t_obs'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            })
            
    


In [46]:
with open('data/kepler_stellar_q17_add_dispersions_reduction_reduction_cool_2020.csv','w') as f:
    s=["kepid",
        "ra",
        "dec",
        "cdpp3",
        "cdpp6",
        "cdpp12",
        'st_quarters',
        't_obs',
        "std_vra",
        "std_vdec",
        "mean_vra",
        "mean_vdec",
        "sigma_vra",
        "sigma_vdec",
        "kepmag",
        "teff",
        "logg",
        "radius",
        "feh",
        "mass",
        "dens",
        "dist",
        "av",
        "jmag",
        "hmag",
        "kmag",
        "ra_gaia",
        "dec_gaia",
        "parallax",
        "parallax_error",
        "pmra",
        "pmra_error",
        "pmdec",
        "pmdec_error",
        "phot_g_mean_mag",
        "teff_b2018",
        "teffe_b2018",
        "rad_b2018",
        "radep_b2018",
        "radem_b2018",
      ]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['sigma_vra'][i]<2 and data['sigma_vdec'][i]<2:
            writer.writerow({"kepid":data['kepid'][i],
                            "ra":data['ra'][i],
                            "dec":data['dec'][i],
                            "cdpp3":data['cdpp3'][i],
                            "cdpp6":data['cdpp6'][i],
                            "cdpp12":data['cdpp12'][i],
                            'st_quarters':data['st_quarters'][i],
                            't_obs':data['t_obs'][i],
                            "std_vra":data['std_vra'][i],
                            "std_vdec":data['std_vdec'][i],
                            "mean_vra":data['mean_vra'][i],
                            "mean_vdec":data['mean_vdec'][i],
                            "sigma_vra":data['sigma_vra'][i],
                            "sigma_vdec":data['sigma_vdec'][i],
                            "kepmag":data['kepmag'][i],
                            "teff":data['teff'][i],
                            "logg":data['logg'][i],
                            "feh":data['feh'][i],
                            "radius":data['radius'][i],
                            "mass":data['mass'][i],
                            "dens":data['dens'][i],
                            "dist":data['dist'][i],
                            "av":data['av'][i],
                            "jmag":data['jmag'][i],
                            "hmag":data['hmag'][i],
                            "kmag":data['kmag'][i],
                            "ra_gaia":data['ra_gaia'][i],
                            "dec_gaia":data['dec_gaia'][i],
                            "parallax":data['parallax'][i],
                            "parallax_error":data['parallax_error'][i],
                            "pmra":data['pmra'][i],
                            "pmra_error":data['pmra_error'][i],
                            "pmdec":data['pmdec'][i],
                            "pmdec_error":data['pmdec_error'][i],
                            "phot_g_mean_mag":data['phot_g_mean_mag'][i],
                            "teff_b2018":data['teff_b2018'][i],
                            "teffe_b2018":data['teffe_b2018'][i],
                            "rad_b2018":data['rad_b2018'][i],
                            "radep_b2018":data['radep_b2018'][i],
                            "radem_b2018":data['radem_b2018'][i],
                            })
            
    


In [47]:
data1=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_hot_2020.csv')
data2=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_cool_2020.csv')
print(len(data1)+len(data2))

113942


In [6]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_hot_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_hot.csv',sep=',')

def N_s(Rp,P,i):
    os.system('mkdir result/N_hot_2020/')
    N_s_temp=0
    with open('result/N_hot_2020/'+str(i)+'.csv','w') as f:
        s=['SNR',
           'fn',
           'feff',]
        writer=csv.DictWriter(f,fieldnames=s)
        writer.writeheader()        
        for i in range(0,len(stellar_table)):
            a_temp=a(stellar_table['mass'][i],P)
            tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
            delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
            n_temp=n(stellar_table['t_obs'][i],P)
            x=[3,6,12]
            y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
            popt, pcov = curve_fit(sigma_s, x, y)
            sigma_LC=popt[0]
            cdppindex=popt[1]
            sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
            SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
            fn_temp=f_n(stellar_table['t_obs'][i],P)
            feff_temp=f_eff(SNR_temp)
            writer.writerow({'SNR':SNR_temp,
                             'fn':fn_temp,
                             'feff':feff_temp,})                             
            N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def N_s2(Rp,P):
#    os.system('mkdir result/N_s/')
    N_s_temp=0
#    with open('result/N_s/'+str(kepid)+'.csv','w') as f:
#        s=['SNR',
#           'fn',
#           'feff',]
#        writer=csv.DictWriter(f,fieldnames=s)
#        writer.writeheader()        
    for i in range(0,len(stellar_table)):
        a_temp=a(stellar_table['mass'][i],P)
        tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
        delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
        n_temp=n(stellar_table['t_obs'][i],P)
        x=[3,6,12]
        y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
        popt, pcov = curve_fit(sigma_s, x, y)
        sigma_LC=popt[0]
        cdppindex=popt[1]
        sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
        SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
        fn_temp=f_n(stellar_table['t_obs'][i],P)
        feff_temp=f_eff(SNR_temp)
#        writer.writerow({'SNR':SNR_temp,
#                         'fn':fn_temp,
#                         'feff':feff_temp,})                             
        N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def f_occurrence_dot(Rp,P,Ms,Rs,e,i):
    N_s_temp=N_s(Rp,P,i)
    a_temp=a(Ms,P)
    fgeo_temp=f_geo(Rp,Rs,a_temp,e)
    f_occurrence_temp=1./(fgeo_temp*N_s_temp)
    return f_occurrence_temp
    

def f_occurrence_cell(Rp_low,Rp_high,P_low,P_high,f_occurrence_everyplanet): # the log base of Rp and P
    count=0
    for i in range(0,len(koi_table)):
        if koi_table['koi_prad'][i]>Rp_low and koi_table['koi_prad'][i]<=Rp_high and koi_table['koi_period'][i]>P_low and koi_table['koi_period'][i]<=P_high:
            count+=1
    
    if count>0:
        f_occurrence_temp=0
        for i in range(0,len(koi_table)):
            if koi_table['koi_prad'][i]>Rp_low and koi_table['koi_prad'][i]<=Rp_high and koi_table['koi_period'][i]>P_low and koi_table['koi_period'][i]<=P_high:
#                N_s_temp=N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i])
#                a_temp=a(koi_table['mass'][i],koi_table['koi_period'][i])
#                fgeo_temp=f_geo(koi_table['koi_prad'][i],koi_table['rad_b2018'][i],a_temp,e)
                if f_occurrence_everyplanet[i]!=np.inf:
                    f_occurrence_temp+=float(f_occurrence_everyplanet[i])
        
        temp=1.0*count/f_occurrence_temp
        if f_occurrence_temp>=1:
            return count,f_occurrence_temp,0
        else:
            deviation_N_s=temp*f_occurrence_temp*(1-f_occurrence_temp)
            std_N_s=deviation_N_s**0.5
            std_cell=std_N_s/temp
            return count,f_occurrence_temp,std_cell
    
    else:
        base_period=5
        base_radius=2
        Rp_center=base_radius**((math.log(Rp_low,base_radius)+math.log(Rp_high,base_radius))/2)
        P_center=base_period**((math.log(P_low,base_period)+math.log(P_high,base_period))/2)
        N_s_temp=N_s2(Rp_center,P_center)
        fgeo_temp=0.1
        if N_s_temp==0:
            f_occurrence_temp=0
        else:
            f_occurrence_temp=1./(fgeo_temp*N_s_temp)
            
        return count,f_occurrence_temp,0


In [4]:
#print(Ns_temp)
f_occurrence_everyplanet=[]
for i in range(0,len(koi_table)):
    P=koi_table['koi_period'][i]
    Rp=koi_table['koi_prad'][i]
    Ms=koi_table['mass'][i]
    Rs=koi_table['rad_b2018'][i]
    a_temp=a(Ms,P)
    N_s_temp=N_s(Rp,P,i)
    fgeo_temp=f_geo(Rp,Rs,a_temp,e)
    f_occurrence_temp=1./(fgeo_temp*N_s_temp)
    f_occurrence_everyplanet.append(f_occurrence_temp)
    print(i,f_occurrence_temp,koi_table['kepid'][i])

0 0.00036921904456304656 5120225
1 0.002736514936873619 10600261
2 0.006117547639227289 10600261
3 0.0006882936559557296 10600261
4 0.0011039521935482103 8414159
5 0.001349478340396299 3554600
6 0.0008251021582124662 5977541
7 0.002326449124901366 6697756
8 0.002421886974592802 9991621
9 0.0014681982472798223 8414716
10 0.001866528327578874 3241557
11 0.0020003631364627376 8042453
12 0.001975092044791658 11298815
13 0.0004851760305003158 10070468
14 0.000637665433074137 3762468
15 0.003235830689191108 8125580
16 0.004677906314278875 6522242
17 0.007656252774598594 10213902
18 0.011763933312168372 9157030
19 0.00320330332764249 7265298
20 0.002436575350364465 7265298
21 0.032177357981495826 10214341
22 0.0020695133672562612 9899256
23 0.0031621707789569343 9730163
24 0.0023732161197780473 9730163
25 0.0017397028622409723 10090257
26 0.0004883897032931379 6312534
27 0.005145510470231995 8494617
28 0.0027967712659890923 3531558
29 0.0030409130646746577 11499263
30 0.0002499287196186915 91

247 0.009717146829381573 7898352
248 0.0035275512073653877 6665064
249 0.00024049749371788035 6665064
250 0.01054275870652197 6504954
251 0.0010992779420822016 9758032
252 0.00025773593901123486 11017094
253 0.0025784803662192908 8230616
254 0.0010929113544936768 8230616
255 0.00037815992980858113 10024051
256 0.002231155937269905 2719873
257 0.0004578195279046085 7200225
258 0.0002651867265401075 2558363
259 0.0036870116594003104 8617363
260 0.0004071712667498883 10661771
261 0.0011933924439587896 9824805
262 0.0027556120939225097 6035124
263 0.0001578817777307618 5636648
264 0.0008822371431105102 9277896
265 0.0011255713881733118 10266860
266 0.04170613010788322 8608544
267 0.04993060227421592 8608544
268 0.003497494641715072 10799735
269 0.00021109148144990534 10485179
270 0.0002044840195388655 10407020
271 0.00026935523869856014 10936786
272 0.02284307498188688 10937029
273 0.0011375667497739636 10937029
274 0.0015315152141321335 8711794
275 0.006504071005823229 9945370
276 0.00147

In [7]:
bins_period=np.logspace(math.log(0.4,5),math.log(250,5),13,base=5)
bins_radius=np.logspace(-1.5,5.5,15,base=2)   

occurrence_cell=np.zeros((14,12))
std_cell=np.zeros((14,12))
count_cell=np.zeros((14,12))
for i in range(0,len(bins_radius)-1):
    for j in range(0,len(bins_period)-1):
        Rp_low=bins_radius[i]
        Rp_high=bins_radius[i+1]
        P_low=bins_period[j]
        P_high=bins_period[j+1]
        temp=f_occurrence_cell(Rp_low,Rp_high,P_low,P_high,f_occurrence_everyplanet)
        count_cell[i,j]=temp[0]
        occurrence_cell[i,j]=temp[1]
        std_cell[i,j]=temp[2]

In [8]:
with open('result/occurrence_rate_hot_cell_2020.csv','w') as f:
    s=['i',
       'j',
       'count_cell',
       'occurrence_cell',
       'std_cell',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(0,len(bins_radius)-1):
        for j in range(0,len(bins_period)-1):
            writer.writerow({'i':i,
                             'j':j,
                             'count_cell':count_cell[i,j],
                             'occurrence_cell':occurrence_cell[i,j],
                             'std_cell':std_cell[i,j]})